##### Copyright 2019 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Text generation with an RNN

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/text/tutorials/text_generation"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/text/blob/master/docs/tutorials/text_generation.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/text/blob/master/docs/tutorials/text_generation.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/text/docs/tutorials/text_generation.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

This tutorial demonstrates how to generate text using a character-based RNN. You will work with a dataset of Shakespeare's writing from Andrej Karpathy's [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/). Given a sequence of characters from this data ("Shakespear"), train a model to predict the next character in the sequence ("e"). Longer sequences of text can be generated by calling the model repeatedly.

Note: Enable GPU acceleration to execute this notebook faster. In Colab: *Runtime > Change runtime type > Hardware accelerator > GPU*.

This tutorial includes runnable code implemented using [tf.keras](https://www.tensorflow.org/guide/keras/sequential_model) and [eager execution](https://www.tensorflow.org/guide/eager). The following is the sample output when the model in this tutorial trained for 30 epochs, and started with the prompt "Q":

<pre>
QUEENE:
I had thought thou hadst a Roman; for the oracle,
Thus by All bids the man against the word,
Which are so weak of care, by old care done;
Your children were in your holy love,
And the precipitation through the bleeding throne.

BISHOP OF ELY:
Marry, and will, my lord, to weep in such a one were prettiest;
Yet now I was adopted heir
Of the world's lamentable day,
To watch the next way with his father with his face?

ESCALUS:
The cause why then we are all resolved more sons.

VOLUMNIA:
O, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, it is no sin it should be dead,
And love and pale as any will to that word.

QUEEN ELIZABETH:
But how long have I heard the soul for this world,
And show his hands of life be proved to stand.

PETRUCHIO:
I say he look'd on, if I must be content
To stay him from the fatal of our country's bliss.
His lordship pluck'd from this sentence then for prey,
And then let us twain, being the moon,
were she such a case as fills m
</pre>

While some of the sentences are grammatical, most do not make sense. The model has not learned the meaning of words, but consider:

* The model is character-based. When training started, the model did not know how to spell an English word, or that words were even a unit of text.

* The structure of the output resembles a play—blocks of text generally begin with a speaker name, in all capital letters similar to the dataset.

* As demonstrated below, the model is trained on small batches of text (100 characters each), and is still able to generate a longer sequence of text with coherent structure.

## Setup

### Import TensorFlow and other libraries

In [1]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

import numpy as np
import os
import time

In [2]:
from names import raw_data, vectorize, seq_model

### Read the data

First, look in the text:

In [3]:
raw = raw_data.load_raw("../data/names.txt")
text = raw_data.join_raw(raw)
vocab = vectorize.extract_vocab(text)

print(f'Length of text: {len(text)} characters')

Length of text: 651954 characters


In [4]:
text[:300]

'Adray Lasbard                                                                                       \nAlbel Nox                                                                                           \nCliff Fittir                                                                                      '

In [5]:
# Take a look at the first 250 characters in text
print(text[:250])

Adray Lasbard                                                                                       
Albel Nox                                                                                           
Cliff Fittir                                    


In [6]:
# The unique characters in the file
vocab = vectorize.extract_vocab(text)
print(f'{len(vocab)} unique characters')

78 unique characters


## Process the text

### Vectorize the text

Before training, you need to convert the strings to a numerical representation. 

The `preprocessing.StringLookup` layer can convert each character into a numeric ID. It just needs the text to be split into tokens first.

In [7]:
example_texts = ['abcdefg', 'xyz']

chars = vectorize.unicode_split(example_texts)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [8]:
vectorator = vectorize.Vectorator(vocab)

It converts form tokens to character IDs:

In [9]:
ids = vectorator.ids_from_chars(chars)
ids

<tf.RaggedTensor [[45, 46, 47, 48, 49, 50, 51], [68, 69, 70]]>

Since the goal of this tutorial is to generate text, it will also be important to invert this representation and recover human-readable strings from it. For this you can use `preprocessing.StringLookup(..., invert=True)`.  

Note: Here instead of passing the original vocabulary generated with `sorted(set(text))` use the `get_vocabulary()` method of the `preprocessing.StringLookup` layer so that the `[UNK]` tokens is set the same way.

This layer recovers the characters from the vectors of IDs, and returns them as a `tf.RaggedTensor` of characters:

In [10]:
chars = vectorator.chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

### The prediction task

Given a character, or a sequence of characters, what is the most probable next character? This is the task you're training the model to perform. The input to the model will be a sequence of characters, and you train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?


### Create training examples and targets

Next divide the text into example sequences. Each input sequence will contain `seq_length` characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of `seq_length+1`. For example, say `seq_length` is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the `tf.data.Dataset.from_tensor_slices` function to convert the text vector into a stream of character indices.

In [11]:
ds = vectorize.DataSet(text)

The `batch` method lets you easily convert these individual characters to sequences of the desired size.

For training you'll need a dataset of `(input, label)` pairs. Where `input` and 
`label` are sequences. At each time step the input is the current character and the label is the next character. 

Here's a function that takes a sequence as input, duplicates, and shifts it to align the input and label for each timestep:

In [12]:
vectorize.split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

### Create training batches

You used `tf.data` to split the text into manageable sequences. But before feeding this data into the model, you need to shuffle the data and pack it into batches.

## Build The Model

This section defines the model as a `keras.Model` subclass (For details see [Making new Layers and Models via subclassing](https://www.tensorflow.org/guide/keras/custom_layers_and_models)). 

This model has three layers:

* `tf.keras.layers.Embedding`: The input layer. A trainable lookup table that will map each character-ID to a vector with `embedding_dim` dimensions;
* `tf.keras.layers.GRU`: A type of RNN with size `units=rnn_units` (You can also use an LSTM layer here.)
* `tf.keras.layers.Dense`: The output layer, with `vocab_size` outputs. It outputs one logit for each character in the vocabulary. These are the log-likelihood of each character according to the model.

In [13]:
# Length of the vocabulary in chars
# vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [14]:
model = seq_model.SeqModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=ds.vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

For each character the model looks up the embedding, runs the GRU one timestep with the embedding as input, and applies the dense layer to generate logits predicting the log-likelihood of the next character:

![A drawing of the data passing through the model](images/text_generation_training.png)

Note: For training you could use a `keras.Sequential` model here. To  generate text later you'll need to manage the RNN's internal state. It's simpler to include the state input and output options upfront, than it is to rearrange the model architecture later. For more details see the [Keras RNN guide](https://www.tensorflow.org/guide/keras/rnn#rnn_state_reuse).

## Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [16]:
for input_example_batch, target_example_batch in ds.dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 79) # (batch_size, sequence_length, vocab_size)


In the above example the sequence length of the input is `100` but the model can be run on inputs of any length:

In [17]:
model.summary()

Model: "seq_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  20224     
_________________________________________________________________
gru (GRU)                    multiple                  3938304   
_________________________________________________________________
dense (Dense)                multiple                  80975     
Total params: 4,039,503
Trainable params: 4,039,503
Non-trainable params: 0
_________________________________________________________________


To get actual predictions from the model you need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary.

Note: It is important to _sample_ from this distribution as taking the _argmax_ of the distribution can easily get the model stuck in a loop.

Try it for the first example in the batch:

In [18]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

This gives us, at each timestep, a prediction of the next character index:

In [19]:
sampled_indices

array([27,  9, 27, 54, 28, 33, 43, 78, 72, 28, 14,  8, 47, 69, 62, 15, 48,
        8, 18, 31, 33, 70, 16, 55, 67, 59, 14, 76, 22,  3, 16,  1, 68, 53,
       78, 68, 35, 59, 61, 64, 65, 32, 77, 32, 21,  4,  8, 24, 27, 11, 41,
       75, 25,  7, 76,  3, 10, 44, 50,  9, 45, 71, 54, 17, 77, 32,  2, 29,
       52, 52, 74, 11,  5,  1, 30, 29, 13, 61, 46, 58, 61, 72, 42, 36,  9,
        6, 49, 48, 52, 46, 16, 35,  2, 35, 27, 25, 10, 61, 11, 15])

Decode these to see the text predicted by this untrained model:

In [31]:
print("Input:\n", ds.text_from_ids(np.array([input_example_batch[0]])))
print()
print("Next Char Predictions:\n", ds.text_from_ids(np.array([sampled_indices])))

Input:
 ['Great Deku Tree                                                                                     ']

Next Char Predictions:
 ["I0IjJOYīáJ6/cyr7d/?MOz8kwo6öD&8\nxiīxQoqtuNāNC'/FI2WóG.ö&1Zf0aÓj9āN Khhí2,\nLK5qbnqáXR0-edhb8Q QIG1q27"]


## Train the model

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

### Attach an optimizer, and a loss function

The standard `tf.keras.losses.sparse_categorical_crossentropy` loss function works in this case because it is applied across the last dimension of the predictions.

Because your model returns logits, you need to set the `from_logits` flag.


In [34]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [35]:
example_batch_loss = loss(target_example_batch, example_batch_predictions)
mean_loss = example_batch_loss.numpy().mean()
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", mean_loss)

Prediction shape:  (64, 100, 79)  # (batch_size, sequence_length, vocab_size)
Mean loss:         4.383516


A newly initialized model shouldn't be too sure of itself, the output logits should all have similar magnitudes. To confirm this you can check that the exponential of the mean loss is approximately equal to the vocabulary size. A much higher loss means the model is sure of its wrong answers, and is badly initialized:

In [36]:
tf.exp(mean_loss).numpy()

80.119225

Configure the training procedure using the `tf.keras.Model.compile` method. Use `tf.keras.optimizers.Adam` with default arguments and the loss function.

In [37]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

Use a `tf.keras.callbacks.ModelCheckpoint` to ensure that checkpoints are saved during training:

In [38]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Execute the training

To keep training time reasonable, use 10 epochs to train the model. In Colab, set the runtime to GPU for faster training.

In [39]:
EPOCHS = 20

In [40]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
100/100 [==============================] - 6s 37ms/step - loss: 0.5424
Epoch 2/20
100/100 [==============================] - 4s 37ms/step - loss: 0.3307
Epoch 3/20
100/100 [==============================] - 4s 37ms/step - loss: 0.2974
Epoch 4/20
100/100 [==============================] - 4s 37ms/step - loss: 0.3061
Epoch 5/20
100/100 [==============================] - 4s 38ms/step - loss: 0.2783
Epoch 6/20
100/100 [==============================] - 4s 38ms/step - loss: 0.2663
Epoch 7/20
100/100 [==============================] - 4s 38ms/step - loss: 0.2655
Epoch 8/20
100/100 [==============================] - 4s 38ms/step - loss: 0.2703
Epoch 9/20
100/100 [==============================] - 4s 38ms/step - loss: 0.2628
Epoch 10/20
100/100 [==============================] - 4s 38ms/step - loss: 0.2632
Epoch 11/20
100/100 [==============================] - 4s 38ms/step - loss: 0.2581
Epoch 12/20
100/100 [==============================] - 4s 38ms/step - loss: 0.2484
Epoch 13/20
1

## Generate text

The simplest way to generate text with this model is to run it in a loop, and keep track of the model's internal state as you execute it.

![To generate text the model's output is fed back to the input](images/text_generation_sampling.png)

Each time you call the model you pass in some text and an internal state. The model returns a prediction for the next character and its new state. Pass the prediction and state back in to continue generating text.


The following makes a single step prediction:

In [41]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [42]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

Run it in a loop to generate some text. Looking at the generated text, you'll see the model knows when to capitalize, make paragraphs and imitates a Shakespeare-like writing vocabulary. With the small number of training epochs, it has not yet learned to form coherent sentences.

In [43]:
start = time.time()
states = None
next_char = tf.constant(['Romeo '])
result = [next_char]

for n in range(100):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Romeo                                                                                                      

________________________________________________________________________________

Run time: 0.804236650466919


The easiest thing you can do to improve the results is to train it for longer (try `EPOCHS = 30`).

You can also experiment with a different start string, try adding another RNN layer to improve the model's accuracy, or adjust the temperature parameter to generate more or less random predictions.

If you want the model to generate text *faster* the easiest thing you can do is batch the text generation. In the example below the model generates 5 outputs in about the same time it took to generate 1 above. 

In [44]:
start = time.time()
states = None
next_char = tf.constant(list("QWERTYUIOPASDFGHJKLZXCVBNM"))
result = [next_char]

for n in range(100):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(
[b'Quheny Iruniaka                                                                                      '
 b'WortG                                                                                                '
 b'Effya Ruseis Sa Tramurt                                                                              '
 b'Ranza                                                                                                '
 b'Tromp Coat                                                                                           '
 b'Yegul Bragh                                                                                          '
 b'Umer Aughterm                                                                                        '
 b'Ispo                                                                                                 '
 b'Ocarige                                                                                              '
 b'Preato Ros                      

## Export the generator

This single-step model can easily be [saved and restored](https://www.tensorflow.org/guide/saved_model), allowing you to use it anywhere a `tf.saved_model` is accepted.

In [45]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')


FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.



FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.


INFO:tensorflow:Assets written to: one_step\assets


INFO:tensorflow:Assets written to: one_step\assets


In [46]:
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

ROMEO:a                                                                                                   


## Advanced: Customized Training

The above training procedure is simple, but does not give you much control.
It uses teacher-forcing which prevents bad predictions from being fed back to the model, so the model never learns to recover from mistakes.

So now that you've seen how to run the model manually next you'll implement the training loop. This gives a starting point if, for example, you want to implement _curriculum  learning_ to help stabilize the model's open-loop output.

The most important part of a custom training loop is the train step function.

Use `tf.GradientTape` to track the gradients. You can learn more about this approach by reading the [eager execution guide](https://www.tensorflow.org/guide/eager).

The basic procedure is:

1. Execute the model and calculate the loss under a `tf.GradientTape`.
2. Calculate the updates and apply them to the model using the optimizer.

In [47]:
class CustomTraining(MyModel):
  @tf.function
  def train_step(self, inputs):
      inputs, labels = inputs
      with tf.GradientTape() as tape:
          predictions = self(inputs, training=True)
          loss = self.loss(labels, predictions)
      grads = tape.gradient(loss, model.trainable_variables)
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

      return {'loss': loss}

The above implementation of the `train_step` method follows [Keras' `train_step` conventions](https://www.tensorflow.org/guide/keras/customizing_what_happens_in_fit). This is optional, but it allows you to change the behavior of the train step and still use keras' `Model.compile` and `Model.fit` methods.

In [48]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [49]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [50]:
model.fit(dataset, epochs=1)

100/100 [==============================] - 6s 40ms/step - loss: 0.5414


Or if you need more control, you can write your own complete custom training loop:

In [51]:
EPOCHS = 30

mean = tf.metrics.Mean()

for epoch in range(EPOCHS):
    start = time.time()

    mean.reset_states()
    for (batch_n, (inp, target)) in enumerate(dataset):
        logs = model.train_step([inp, target])
        mean.update_state(logs['loss'])

        if batch_n % 50 == 0:
            template = f"Epoch {epoch+1} Batch {batch_n} Loss {logs['loss']:.4f}"
            print(template)

    # saving (checkpoint) the model every 5 epochs
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print()
    print(f'Epoch {epoch+1} Loss: {mean.result().numpy():.4f}')
    print(f'Time taken for 1 epoch {time.time() - start:.2f} sec')
    print("_"*80)

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 0.3546
Epoch 1 Batch 50 Loss 0.3626

Epoch 1 Loss: 0.3432
Time taken for 1 epoch 4.82 sec
________________________________________________________________________________
Epoch 2 Batch 0 Loss 0.3352
Epoch 2 Batch 50 Loss 0.3490

Epoch 2 Loss: 0.2995
Time taken for 1 epoch 4.30 sec
________________________________________________________________________________
Epoch 3 Batch 0 Loss 0.3017
Epoch 3 Batch 50 Loss 0.3941

Epoch 3 Loss: 0.3002
Time taken for 1 epoch 4.26 sec
________________________________________________________________________________
Epoch 4 Batch 0 Loss 0.3002
Epoch 4 Batch 50 Loss 0.2861

Epoch 4 Loss: 0.2766
Time taken for 1 epoch 4.24 sec
________________________________________________________________________________
Epoch 5 Batch 0 Loss 0.2230
Epoch 5 Batch 50 Loss 0.2779

Epoch 5 Loss: 0.2808
Time taken for 1 epoch 4.45 sec
________________________________________________________________________________
Epoch 6 Batch 0 Loss 0.2855
Epoch 6 Batc

In [52]:
start = time.time()
states = None
next_char = tf.constant(list("QWERTYUIOPASDFGHJKLZXCVBNM"))
result = [next_char]

for n in range(40):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(
[b'Quinfair                                 '
 b'Wanth                                    '
 b'Ellia                                    '
 b'Runa                                     '
 b'Torby                                    '
 b'Yug Parinz                               '
 b'Unsearie                                 '
 b'Izri                                     '
 b'Olia Jelfa                               '
 b'Pokielta                                 '
 b'Anne                                     '
 b'Sah Larabold                             '
 b'Degame Lukis Caelum                      '
 b'Futo                                     '
 b'Griff                                    '
 b'Hash                                     '
 b'Jyon Drimam                              '
 b"Ko'shoho                                 "
 b'Liqus                                    '
 b'Zolb Broh                                '
 b'Xansurabueq                              '
 b'Choco               

In [53]:
start = time.time()
states = None
next_char = tf.constant(list("QWERTYUIOPASDFGHJKLZXCVBNM"))
result = [next_char]

for n in range(40):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
result = [" ".join([a.strip() for a in name.decode("utf-8").split() if len(a)]) for name in result.numpy()]
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

['Quan', 'Wycch', 'Exfis Nohder', 'Rinaha', 'Tyrit', 'Yughin', 'Uhniriok Oad Koropa', 'Igona Shimis', 'Ovagri', 'Poppi', 'A-Tappa', 'Savine', 'Dumah Jaabu do', 'Faketh', 'GrannyWa', 'Halzand', 'Jowanne', "Ke'n jah", 'Leviar Luki', 'Zhani', 'Xundler', 'Cleus Fate', 'Viwela', 'Bgend Lacheery', 'Noraha', 'Masper'] 

________________________________________________________________________________

Run time: 0.08176589012145996


In [54]:
def generate_names(stem="", n=1):
    names = []
    for _ in range(n):
        states = None
        next_char = [stem + ch for ch in "QWERTYUIOPASDFGHJKLZXCVBNM"]
        next_char = tf.constant(next_char)
        result = [next_char]

        for n in range(40):
          next_char, states = one_step_model.generate_one_step(next_char, states=states)
          result.append(next_char)

        result = tf.strings.join(result)
        names += [" ".join([a.strip() for a in name.decode("utf-8").split() if len(a)]) for name in result.numpy()]
    return names

In [55]:
generate_names(n=10)

['Qunan',
 'Wattitahf',
 'Eddrate',
 'Rowamal',
 'Tepla Totorien',
 'Yeski',
 'Unhweaner',
 'Ictina',
 'Old Mah',
 'Puncit',
 'Azer',
 'Soutasner',
 'Dr. Tradsei',
 'Frana',
 'Gaurthellerid',
 'Heschov',
 'Jecion',
 'Kutoka',
 "Letri's Bar Les Soum",
 'Zekkrim',
 'Xhia Vectrima',
 'Chloh',
 'Vannion',
 'Bakiba',
 'Nubrokuix',
 'Mikkue',
 'Quiltous of Ciant Fa',
 'Waltzon',
 'Enneus',
 'Rowdelpus',
 'Theit',
 'Yogenkt',
 'Ukaagale',
 'Illah',
 'Oad',
 'Phoco',
 'Anigagu',
 'Stahny',
 'Derolicite',
 'Fildie Drien',
 'Gairo',
 'Hathia Leffall',
 'Jew-O-Senne',
 'Katara Yoto',
 'Lithimo',
 'Zusanaru',
 'Xard-Chondian',
 'Cain Leugh',
 'Vaevan chep',
 'Bicha',
 'Nalta',
 'Modori',
 'Quiek trick',
 'Westar',
 'Eimiso Sabronaulur',
 "Raine's Lother",
 'Thegnagl',
 'Yukh',
 'Utitona',
 'Ilake Selkerkiee',
 'Orolia Arbirchina',
 'Partos',
 'Antconburet de Houshairle',
 'Sairomaha',
 'Dinkle',
 'Fostorr Yosser',
 'Guiri ofise Thie',
 'Harthon',
 'Jopeen',
 "Ko'shomama",
 'Linduv',
 'Zerlance',
 

In [56]:
generate_names('Joby of ', n=10)

['Joby of Que perelle Feemirot/',
 'Joby of Warronos Orchors',
 'Joby of Eltheim',
 'Joby of Rundos',
 'Joby of Tinne',
 'Joby of Yuyme',
 'Joby of Uldharm',
 'Joby of Iughmind',
 'Joby of Order',
 'Joby of Phiden',
 'Joby of Azentom',
 'Joby of Selma',
 'Joby of Deagen',
 'Joby of Farremard',
 'Joby of Gurdan',
 'Joby of Heni of',
 'Joby of Jeven',
 'Joby of Krand',
 'Joby of Lewmaldr',
 'Joby of Zukeny Selio',
 'Joby of XII',
 'Joby of Chovencel',
 'Joby of Vex',
 'Joby of Beemburd',
 'Joby of N.',
 'Joby of Miss',
 'Joby of Queen',
 'Joby of Wink',
 'Joby of Elman',
 'Joby of RedUg',
 'Joby of Ter',
 'Joby of Yenne',
 'Joby of Unnerd',
 'Joby of Ihmovield',
 'Joby of Oandere d',
 'Joby of Phigeshon',
 'Joby of Aunchtion',
 'Joby of Scima',
 'Joby of Delef',
 'Joby of Furlemerm',
 'Joby of Gyrld',
 'Joby of Heurion',
 'Joby of Jesherk',
 'Joby of Keep Sal Eroda',
 'Joby of Leshaille',
 'Joby of Zliepos',
 'Joby of XII',
 'Joby of Crespann',
 'Joby of Vandredde',
 'Joby of Bahmell',
 

In [57]:
generate_names('Joby the ')

['Joby the Queen',
 'Joby the Worrina',
 'Joby the Erondour',
 'Joby the Rabe',
 'Joby the Tazephien',
 "Joby the Ye'nard Ma",
 'Joby the Urage',
 'Joby the Iglandsorne',
 'Joby the Oakville',
 'Joby the Prepesonord',
 'Joby the Aleghring',
 'Joby the Sepersant',
 'Joby the Daraldi',
 'Joby the Fabrongo',
 'Joby the Grade',
 'Joby the Hugeh',
 'Joby the Jreane',
 'Joby the Korten',
 'Joby the Lougis',
 'Joby the Zadges',
 'Joby the Xokkhail',
 'Joby the Chamelka',
 'Joby the Vaimignaix',
 'Joby the Brinpe',
 'Joby the Naighessa',
 'Joby the Maske']

In [58]:
place = []
for name in generate_names():
    name = name.split()[0]
    for long_name in generate_names(name + " of "):
        print(long_name)
        place.append(long_name.split()[2])

Quata of Quen
Quata of Watch
Quata of E-Th
Quata of Rur Bus
Quata of Teema
Quata of Yusair
Quata of Urmukk
Quata of IV
Quata of Orumust
Quata of Pilmurl
Quata of Arlmuttre
Quata of Sajudo
Quata of Drid Tailam
Quata of Fat at She
Quata of Gayu
Quata of Hasulfram
Quata of Jea
Quata of Kia
Quata of Lumura
Quata of Zelch
Quata of XVII
Quata of Carlumpe
Quata of Vather
Quata of Bruththin
Quata of Neku
Quata of Mamuri
Wimero of Qeyen
Wimero of Waldarro
Wimero of Ebenour
Wimero of Ra'Ohand
Wimero of Tuner
Wimero of Yanbackory
Wimero of Ulmaldo
Wimero of IXIcharias
Wimero of Ome
Wimero of Pupita
Wimero of Aulmbals
Wimero of Sabont of
Wimero of Delacha
Wimero of Fornama
Wimero of Gagralo
Wimero of Heulbgiclo
Wimero of Jeen
Wimero of Kirasus
Wimero of Lasserkain
Wimero of Zarda
Wimero of Xanrih
Wimero of Camalyon
Wimero of VIv Minea
Wimero of Bamuda
Wimero of Naggosso
Wimero of Mishuld
Emperan of Queen
Emperan of Wernel Celemobyr
Emperan of Elsmild
Emperan of Rewlur
Emperan of Trever
Emperan of 

Cidagolard of Queenne
Cidagolard of Wernny
Cidagolard of Eelum
Cidagolard of Raim Pansa
Cidagolard of Tumov
Cidagolard of Yaggsss de Bairumas
Cidagolard of Ular
Cidagolard of IVdiccanar
Cidagolard of Orbolph
Cidagolard of Prouges
Cidagolard of Allumo
Cidagolard of Sellurd
Cidagolard of Dalmacion
Cidagolard of Futrigean
Cidagolard of Guond
Cidagolard of Hilgasi
Cidagolard of Juges
Cidagolard of Kubor
Cidagolard of Lugglair
Cidagolard of Zuburns
Cidagolard of XII
Cidagolard of Cresant XVII
Cidagolard of V
Cidagolard of Bullugin
Cidagolard of Naugea
Cidagolard of Mangesgers
Viloteen of Queen
Viloteen of Wirne
Viloteen of Elstullv
Viloteen of Re Hana
Viloteen of Tleura
Viloteen of Youits
Viloteen of Ulmari
Viloteen of IV
Viloteen of Ool
Viloteen of Puntestaix
Viloteen of Allearvine
Viloteen of Sartas
Viloteen of Daine
Viloteen of Fenbarce
Viloteen of Grove
Viloteen of Hotrea
Viloteen of Jiens
Viloteen of Kant
Viloteen of Lucure
Viloteen of Zeedare Fe
Viloteen of XIV
Viloteen of Cidiansss
V

In [59]:
place

['Quen',
 'Watch',
 'E-Th',
 'Rur',
 'Teema',
 'Yusair',
 'Urmukk',
 'IV',
 'Orumust',
 'Pilmurl',
 'Arlmuttre',
 'Sajudo',
 'Drid',
 'Fat',
 'Gayu',
 'Hasulfram',
 'Jea',
 'Kia',
 'Lumura',
 'Zelch',
 'XVII',
 'Carlumpe',
 'Vather',
 'Bruththin',
 'Neku',
 'Mamuri',
 'Qeyen',
 'Waldarro',
 'Ebenour',
 "Ra'Ohand",
 'Tuner',
 'Yanbackory',
 'Ulmaldo',
 'IXIcharias',
 'Ome',
 'Pupita',
 'Aulmbals',
 'Sabont',
 'Delacha',
 'Fornama',
 'Gagralo',
 'Heulbgiclo',
 'Jeen',
 'Kirasus',
 'Lasserkain',
 'Zarda',
 'Xanrih',
 'Camalyon',
 'VIv',
 'Bamuda',
 'Naggosso',
 'Mishuld',
 'Queen',
 'Wernel',
 'Elsmild',
 'Rewlur',
 'Trever',
 'Yonnu',
 'Undyriem',
 'IVicheraim',
 'Ohberron',
 'Prenntyte',
 'Almaso',
 'Serl',
 'Dernel',
 'Frunhart',
 'Grands',
 'Haildruse',
 'Jee',
 'Keen',
 'Lishersar',
 'Zar',
 'XVirenarte',
 'Cheviaut',
 'VIdou',
 'Bunremus',
 'Nox',
 'Messearca',
 'Quinz',
 'Warres',
 'Emmiro',
 'Resm',
 'Timilo',
 'Yanguh',
 'Ullurd',
 'Ironhearts',
 'Ommind',
 'Porgais',
 'Amburobos

In [60]:
G = list(filter(lambda name: name[0] == "G", place))
G.sort()
G

['Gagralo',
 'Gaidor',
 'Gamnard',
 'Gardo',
 'Gayu',
 'Gimper',
 'Go',
 'Grands',
 'Grove',
 'Gruma',
 'Guadre',
 'Guart',
 'Guasomo',
 'Guat/Fanry',
 'Guck',
 'Gudgo',
 'Gueldros',
 'Gueull',
 'Guond',
 'Gurde',
 'Gurdus',
 'Gurtmain',
 'Gurva',
 'Guuruh',
 'Guyer',
 'Gylla']

In [61]:
np.random.choice(place, size=10, replace=False)

array(['Verra', 'Bonnein', 'Zertt', 'Prumpant', 'Ulmmurd', 'Preip',
       'Ordor', 'Luveir', 'Jeen', 'Jerne'], dtype='<U14')

In [62]:
np.random.choice(place, size=10, replace=False)

array(['Laiesm', 'Iuder', 'Zern', 'Bartomi', 'Striff', 'Izokit', 'Zezo',
       'Tleura', 'Sager', 'Ya'], dtype='<U14')

In [63]:
np.random.choice(place, size=10, replace=False)

array(['Older', 'Zerne', 'Teer', 'Burdler', 'Wramer', 'Nyle', 'Raset',
       'Sartas', 'IV', 'Kubor'], dtype='<U14')

In [64]:
generate_names()

['Quina Tros',
 'Widra',
 "Edva'virannar",
 'Rapher',
 'Thorl',
 'Yutah',
 'Ulmitothare',
 'Indgiel',
 'Oe',
 'Pitisa',
 'Amph',
 'Scarice',
 'Dr.Sphrader',
 'F. Chonnshon',
 'Gleaddring',
 'Hancel',
 'Jell Andoribloy',
 'Kurel Aphuralt',
 'Lunel',
 'Zelika',
 'Xermaur',
 'Chulor',
 'Vaiora',
 'Begra Aswerganch',
 'Naki',
 'Maitha']